In [1]:
import pandas as pd
import numpy as np
import datetime
from itertools import product
from scipy.sparse.linalg import svds

#from surprise import Reader, Dataset, evaluate, print_perf, GridSearch
#from surprise import SVD, SVDpp, Prediction, accuracy
#from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('data_engineered_features500.csv', nrows=60018)
#data = data.query("userid == ['user_000001','user_000002','user_000003','user_000004','user_000005']")

data.rename(columns={'track-name':'track',
                     'artist-name':'artist',
                     'track-total-count':'trackcount',
                     'track-weekday-daytime-count':'trackweekdaydaytimecount',
                     'artist-total-count':'artistcount',
                     'artist-weekday-daytime-count':'artistweekdaydaytimecount'}, inplace=True)

data.drop(['timestamp', 'songlength', 'last-seen-song', 'last-seen-artist', 'gender', 'age'], axis=1, inplace=True)
data = data[[c for c in data if c not in ['skipped']] + ['skipped']]

In [3]:
print(len(data))
data.dtypes

60018


userid                       object
track                        object
artist                       object
weekday                       int64
hour                          int64
weekend                       int64
daytime                       int64
trackcount                    int64
trackweekdaydaytimecount      int64
month                         int64
quarter                       int64
artistcount                   int64
artistweekdaydaytimecount     int64
skipped                       int64
dtype: object

In [4]:
df = data[['userid', 'track', 'artist', 'weekend', 'skipped']].copy()

# merge track and artist to one column, so that
# when doing combinations, we don't have every track with every artist
df['trackartist'] = df['track']+'___'+df['artist']
df.drop(['track', 'artist'], axis=1, inplace=True)

# find rate at which users skip songs by weekend status
df = df.groupby(['userid', 'trackartist', 'weekend'], as_index=False).mean()

In [23]:
# create combinations
# need to remove duplicates to have only distinct combinations
temp = pd.DataFrame(list(product(df['userid'].drop_duplicates(),
                                 df['trackartist'].drop_duplicates(),
                                 df['weekend'].drop_duplicates())),
                   columns = ['userid', 'trackartist', 'weekend'])
len(temp)

24500

In [21]:
# create dense matrix with all combinations
dense_data = temp.merge(df, how='left',
                      on=['userid', 'trackartist', 'weekend'])

# create indicator variable to signify whether a value was imputed
dense_data['missing'] = dense_data['skipped'].isnull().astype('int')

# impute missing values
dense_data = dense_data.fillna(1)

# delete temp dataframe to clear space
del temp

In [22]:
dense_data.describe()

,weekend,skipped,missing
count,24500.00000,24500.000000,24500.000000
mean,0.50000,0.805027,0.801265
std,0.50001,0.392899,0.399056
min,0.00000,0.000000,0.000000
25%,0.00000,1.000000,1.000000
50%,0.50000,1.000000,1.000000
75%,1.00000,1.000000,1.000000
max,1.00000,1.000000,1.000000


In [11]:
from scipy.linalg import svd

u, s, v = svd(dense_data, full_matrices=False)
X = u.dot(np.diag(s))

ValueError: object arrays are not supported